<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Hyperparameter Tuning

## *Data Science Unit 4 Sprint 2 Assignment 4*

## Your Mission, should you choose to accept it...

To hyperparameter tune and extract every ounce of accuracy out of this telecom customer churn dataset: <https://drive.google.com/file/d/1dfbAsM9DwA7tYhInyflIpZnYs7VT-0AQ/view> 

## Requirements

- Load the data
- Clean the data if necessary (it will be)
- Create and fit a baseline Keras MLP model to the data.
- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters
 
 Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?


In [1]:
pip install keras

Note: you may need to restart the kernel to use updated packages.


In [69]:
import pandas as pd
import numpy as np
import sklearn
import keras
from sklearn.model_selection import GridSearchCV, train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

In [3]:
# Load the data
df = pd.read_csv('./WA_Fn-UseC_-Telco-Customer-Churn.csv')

In [4]:
# Clean the data; df.Churn is y

df.isna().sum()
df.isnull().sum()
print(df.shape, df.columns)
df.head()

# Looking at a categorical (binary) problem -> sigmoid activation function

(7043, 21) Index(['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents',
       'tenure', 'PhoneService', 'MultipleLines', 'InternetService',
       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
       'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
       'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn'],
      dtype='object')


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [5]:
# Identify unusable features and refine dataset to address outliers

# Unusable features: 
#  customerID, 

df.describe()
# Outliers: refer to tenure and MonthlyCharges, drop > 2 std-steps from median

df.nunique()

customerID          7043
gender                 2
SeniorCitizen          2
Partner                2
Dependents             2
tenure                73
PhoneService           2
MultipleLines          3
InternetService        3
OnlineSecurity         3
OnlineBackup           3
DeviceProtection       3
TechSupport            3
StreamingTV            3
StreamingMovies        3
Contract               3
PaperlessBilling       2
PaymentMethod          4
MonthlyCharges      1585
TotalCharges        6531
Churn                  2
dtype: int64

In [6]:
def wrangle(X): 
    """ function to wrangle/pre-process the data in df """
    X = X.copy()
    
    # feature engineer: add potentially useful features
    
    # drop unusable features
    X = X.drop(columns = ['customerID','TotalCharges'])
    
    # managing outliers
    X = X.query('tenure < 79')
    X = X.query('MonthlyCharges < 120')
    X = X.query("TechSupport != 'No internet service'")
    
    # encode to categorical 
    X = X.replace({'Yes': 1, 'No': 0,
                 'No phone service': 2,
                 'Month-to-month': 0, 
                  'One year': 1,
                 'Two year': 2, 'Male': 0,
                  'Female': 1, 'Fiber optic': 0,
                  'DSL': 1, 'Electronic check': 0,
                  'Bank transfer (automatic)': 1,
                  'Credit card (automatic)': 2,
                  'Mailed check': 3})
    
    X['tenure'] = pd.cut(X.tenure, 5, labels = [1,2,3,4,5])
    X['MonthlyCharges'] = pd.cut(X.MonthlyCharges, 5, 
                                labels = [1, 2, 3, 4, 5])

    # drop any remaining rows/entries containing one or more nan
    X = X.dropna()
    X = X.astype('float64')
    
    return X

df2 = wrangle(df)

In [7]:
df2.PaymentMethod.value_counts()

0.0    2243
1.0    1212
2.0    1191
3.0     871
Name: PaymentMethod, dtype: int64

In [56]:
# Instantiate X and y
target = 'Churn'
y = df2[[target]].astype('bool')
X = df2.drop(columns = target)

In [57]:
# Get the categorical baseline
y.Churn.value_counts(normalize=True) 

# The goal is set: achieve a 99.9% accuracy;
#  a 26 point improvement over the baseline

False    0.681711
True     0.318289
Name: Churn, dtype: float64

In [58]:
# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(4413, 18) (4413, 1) (1104, 18) (1104, 1)


In [59]:
# Convert to array values
X_train_array = np.asarray(X_train)
X_test_array = np.asarray(X_test)

y_train_array = np.asarray(y_train)
y_test_array = np.asarray(y_test)

assert(X_train.values == X_train_array).all
print(X_train.values)

[[1. 0. 1. ... 0. 1. 4.]
 [0. 0. 0. ... 1. 0. 4.]
 [1. 0. 0. ... 1. 1. 4.]
 ...
 [1. 0. 1. ... 1. 2. 5.]
 [0. 0. 0. ... 0. 3. 2.]
 [1. 0. 1. ... 0. 2. 1.]]


In [60]:
import tensorflow as tf

In [100]:
#Frame the model

# Plant a random seed for reproducibility
seed = 1
np.random.seed(seed)

# Function to create model, required for KerasClassifier
def create_model():
    # create model
    model = Sequential()
    model.add(Dense(24, input_dim=18, activation='sigmoid'))
    model.add(Dropout(rate = .3))
    model.add(Dense(12, activation='sigmoid'))
#     model.add(Dropout(rate = .3))
#     model.add(Dense(6, activation='sigmoid'))
#     model.add(Dropout(rate = .1))
    model.add(Dense(1, activation='sigmoid'))
    
    # Compile model
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    return model

# X = X_train
# y = y_train

# y_train =  keras.utils.to_categorical(y_train, 2)
# y_test = keras.utils.to_categorical(y_test, 2)

# create model
model = KerasClassifier(build_fn=create_model, verbose=0)

# define the grid search parameters; Bests: batch_size = 31, epochs = 30
param_grid = {'batch_size': [31],  
              'epochs': [30]
             } # These are the parameters that can be passed into the .fit() function:
#               fit(x=None, y=None, batch_size=None, epochs=1, verbose=1, callbacks=None, 
#                  validation_split=0.0, validation_data=None, shuffle=True, class_weight=None,
#                 sample_weight=None, initial_epoch=0, steps_per_epoch=None, validation_steps=None,
#                validation_freq=1, max_queue_size=10, workers=1, use_multiprocessing=False)


# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
#grid_result = grid.fit(tf.convert_to_tensor(X_train.values, dtype=tf.float64),
#                      tf.convert_to_tensor(y_train.values, dtype=tf.float64))
#grid_result = grid.fit(tf.convert_to_tensor(X.values, dtype = tf.float64),
#                        tf.convert_to_tensor(y.values, dtype = tf.float64))

grid_result = grid.fit(X_train.values, y_train.values)

# Report Results - best accuracy = .769
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}") 

Best: 0.7675050894419352 using {'batch_size': 31, 'epochs': 30}
Means: 0.7675050894419352, Stdev: 0.004543377432171473 with: {'batch_size': 31, 'epochs': 30}


## Stretch Goals:

- Try to implement Random Search Hyperparameter Tuning on this dataset
- Try to implement Bayesian Optimiation tuning on this dataset using hyperas or hyperopt (if you're brave)
- Practice hyperparameter tuning other datasets that we have looked at. How high can you get MNIST? Above 99%?
- Study for the Sprint Challenge
 - Can you implement both perceptron and MLP models from scratch with forward and backpropagation?
 - Can you implement both perceptron and MLP models in keras and tune their hyperparameters with cross validation?